In [32]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../..')

In [66]:
import deepmatcher as dm
import pandas as pd
from utils.dataset_parser import generate_unlabeled,dropTokensInColumns
import itertools
from utils.deepmatcher_utils import wrapDm

In [4]:
DATA_DIR = '../../datasets/Structured/Beer/'
tableA = pd.read_csv(DATA_DIR+'tableA.csv')
tableB = pd.read_csv(DATA_DIR+'tableB.csv')

In [20]:
def containTokens(attribute,tokens):
    attr_tokens = set(map(lambda t:t.lower(),attribute.split()))
    return set(tokens).issubset(attr_tokens)

In [26]:
def filterTable(table,attribute,tokens):
    filter_row = list(map(lambda attr:containTokens(attr,tokens),table[attribute].values))
    return table[filter_row].copy()

In [30]:
filteredTableA = filterTable(tableA,'Beer_Name',['imperial','red','ale'])
filteredTableB = filterTable(tableB,'Beer_Name',['imperial','red','ale'])
len(filteredTableA),len(filteredTableB)

(58, 47)

In [48]:
unlabeled_ids = list(itertools.product(filteredTableA.id.values,filteredTableB.id.values))
unlabeled_df = pd.DataFrame(unlabeled_ids,columns=['ltable_id','rtable_id'])
unlabeled_df.to_csv(DATA_DIR+'unlabeled_ids.csv',index=False)

In [49]:
unlabeled_values = generate_unlabeled(DATA_DIR,'unlabeled_ids.csv')

In [50]:
unlabeled_values.to_csv(DATA_DIR+'unlabeled.csv',index=False)

In [67]:
unlabeled_without_pattern = dropTokensInColumns(unlabeled_values,['ltable_Beer_Name','rtable_Beer_Name'],
                                               ['imperial','red','ale'])

In [69]:
model_hybrid = dm.MatchingModel(attr_summarizer='hybrid')
model_hybrid.load_state('../../models/beer_hybrid.pth')
model_hybrid = model_hybrid.to('cuda')

In [75]:
model_rnn = dm.MatchingModel(attr_summarizer='rnn')
model_rnn.load_state('../../models/beer_rnn.pth')
model_rnn = model_hybrid.to('cuda')

In [79]:
prediction_rnn = wrapDm(unlabeled_values,model_rnn,outputAttributes=True)

In [80]:
prediction_rnn[(prediction_rnn.match_score>0.5)]

,match_score,ltable_Beer_Name,ltable_Brew_Factory_Name,ltable_Style,ltable_ABV,rtable_Beer_Name,rtable_Brew_Factory_Name,rtable_Style,rtable_ABV
id,,,,,,,,,
